<div style='background-image: url("./title01.png") ; padding: 0px ; background-size: cover ; border-radius: 5px ; height: 200px'>
<div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.7) ; width: 50% ; height: 150px">
<div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.8) ; line-height: 100%">Computers, Waves, Simulations</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.5)">The Finite Element Method - Static Elasticity</div>
        </div>
    </div>
</div>

This notebook covers the following aspects:
* Define the concept of static elasticity 
* Solution using finite element scheme
* Solution using the finite difference scheme and its comparion with FEM solution


## Basic Equations

Static elasticity can be considered a particular case derived from the elastic wave equation when the displacement does not depend on time, i.e $\partial_t^2 u(x,t) = 0$. Under this assumption and departing from the 1D elastic wave equation, the differential equation turns into the Poisson equation

\begin{equation}
-\mu \partial_x^2 u = f,
\end{equation}

where $\mu$ is the shear modulus for a homogeneous media, $u$ is the displacement field, and $f$ is the external force. The solution for this problems is found after bringing this equation into its weak form, applying the free boundary condition, and using the Galerkin principle with a suitable basis. Then, the displacement defined in a discrete set of points $x_i$ is given as the solution of a system of N equations, with 

\begin{equation}
\mathbf{u} = (\mathbf{K}^{T})^{-1} \mathbf{f}
\end{equation}

where $\mathbf{K}$ is the stiffness matrix. For an elastic physical system with constant shear modulus $\mu$ and uniform element size $h$, it is given as

\begin{equation}
K_{ij} = \frac{\mu}{h}
 \begin{pmatrix}
   1 & -1 &    &    & \\
  -1 &  2 & -1 &    & \\
     &    & \ddots  &  &  \\
     &    & -1 &  2 & -1   \\
     &    &    & -1 &  1
 \end{pmatrix}
\end{equation}

The purpose of this notebook is to illustrate how the problem of static elasticity is solved with the finite- element method. We also compare the solution using finite - differences, the so-called relaxation method given by (derivation in the book):

\begin{equation}
u_{i}^{k+1} = \dfrac{u_{i+1}^{k} + u_{i-1}^{k}}{2} + \dfrac{h^2}{2 \mu}f_i
\end{equation}

In [ ]:
# This is a configuration step for the exercise. Please run it before plotting!
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

### 1.  Finite - Element solution

In [ ]:
# ---------------------------------------------------------------
# Initialization of setup
# ---------------------------------------------------------------
nx = 20              # Number of element boundary points
u  = np.zeros(nx)    # Solution vector 
f  = np.zeros(nx)    # Source vector 
mu = 1               # Constant shear modulus 

# Element boundary points
x = np.linspace(0, 1, nx)  # x in [0,1]
h = x[2] - x[1]            # Constant element size

# ---------------------------------------------------------------
# Assemble stiffness matrix K_ij
# ---------------------------------------------------------------
K = np.zeros((nx, nx))
for i in range(1, nx-1):
    for j in range(1, nx-1):
        if i == j:
            K[i, j] = 2*mu/h
        elif i == j + 1:
            K[i, j] = -mu/h
        elif i + 1 == j:
            K[i, j] = -mu/h
        else:
            K[i, j] = 0

# ---------------------------------------------------------------
# Souce term is a spike at i = 3*nx/4
f[int(3*nx/4)] = 1

# Boundary condition at x = 0
u[0] = 0.15 ; f[1] = u[0]/h

# Boundary condition at x = 1
u[nx-1] = 0.05 ; f[nx-2] = u[nx-1]/h

# ---------------------------------------------------------------
# Finite element solution
# ---------------------------------------------------------------
u[1:nx-1] = np.linalg.inv(K[1:nx-1, 1:nx-1]) @ np.transpose(f[1:nx-1]) 

# ---------------------------------------------------------------
# Plotting section
# ---------------------------------------------------------------
plt.figure(figsize=(8,6))

xfe = u 
plt.plot(x, xfe, color='r', lw=2, label='Finite elements')
plt.title('Static Elasticity', size=16)
plt.ylabel('Displacement $u(x)$', size=16)
plt.xlabel('Position $x$', size=16)
plt.axis([0, 1, 0.04, .28])
plt.legend()
plt.grid(True)
plt.show()

### 2. Finite - Difference solution

In [ ]:
# Poisson's equation with relaxation method
# ---------------------------------------------------------------
nt = 500     # Number of iterations
idisp = 2   # Snapshot frequency

# non-zero boundary conditions
u  = np.zeros(nx)   # set u to zero
du = np.zeros(nx)   # du/dx
f  = np.zeros(nx)   # forcing

f[int(3*nx/4)] = 1./h

xfd = np.arange(0, nx)*h

# ---------------------------------------------------------------
# Initialize animated plot
# ---------------------------------------------------------------
fig1, ax1 = plt.subplots(figsize=(8,6))

line1, = ax1.plot(x, xfe, color='r', lw=2, label='FE') 
line2, = ax1.plot(xfd, u, color='k', ls='-.', label='FD relaxation')
ax1.set_title('Static Elasticity with relaxation method', size=16)
ax1.set_ylabel('Displacement, $u$', size=12)
ax1.set_xlabel('Position, $x$', size=12)
ax1.legend(loc=4)
ax1.grid(True)

plt.show()

# ---------------------------------------------------------------
# Initialize boundaries for first time step
u[0] = 0.15    # Boundary condition at x=0
u[nx-1] = 0.05 # Boundary condition at x=1

# Progress Bar
progress_handler = ProgressBarHandler(nt, "Static Elasticity with relaxation method")

# Simulation Results
u_results = np.zeros((math.ceil(nt/idisp), nx))

for it in range(nt):
    # Calculate the average of u (omit boundaries)
    for i in range(1, nx-1):
        du[i] =u [i+1] + u[i-1]
    u = 0.5*( f*h**2/mu + du )
    u[0] = 0.15    # Boundary condition at x=0
    u[nx-1] = 0.05 # Boundary condition at x=1
    fd = u
    
    # Save Results
    if it % idisp == 0:
        u_results[int(it/idisp)] = fd

    progress_handler(it)

None

In [ ]:
from animation import create_animation_fe_static_elasticity

ani = create_animation_fe_static_elasticity(locals())

display(HTML(ani.to_jshtml()))

plt.close()